# Varying the second characteristic decay time in a viscoelastic model based on a Kelvin model

In [1]:
import os
import sys
from datetime import datetime
import numpy as np
import torch

sys.path.append('../src')
from deepymod_torch.DeepMod import DeepMoD
from deepymod_torch.library_function import stress_input_library
import deepymod_torch.VE_datagen as VE_datagen
import deepymod_torch.VE_params as VE_params

/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

Here, we set many parameters that define our problem. We define:
- The shape, frequency and magnitude of the sinc input manipulation
- The model and model type (which in the case that input_type is stress is a Kelvin type)
- The sampling rate from the data that will be generated

In [2]:
omega = 1
E = [1, 1, 1]
eta = [2.5, 1]
input_expr = lambda t: np.sin(t)/(t)
dsigma = lambda t: (1/t)*(np.cos(t) - np.sin(t)/(t))
input_torch_lambda = lambda t: torch.sin(t)/(t)
input_type = 'Stress'
func_desc = 'Sinc'
number_of_samples = 1000
investigated_param = 'Decay Constant 2'

To generate data, we need to choose where and when to evaluate the target data fr a given manipulation. Below, we choose those time points.

In [3]:
time_array = np.linspace(0.00001, 30, 5000)

Next we configure DeepMoD. We configure:
- The initial L1 regularisation penalty
- The number of epochs at each stage of training
- The size and shape of the network
- The library function for calculating potential terms relevant to VE problems.

In [4]:
optim_config = {'lambda': 10**-6, 'max_iterations': 100001, 'final_run_iterations': 10001}
network_config = {'input_dim': 1, 'hidden_dim': 40, 'layers': 5, 'output_dim': 1}
lib_config = {'type': stress_input_library, 'diff_order': 3, 'coeff_sign': 'positive', 'input_type': input_type, 'input_expr': input_torch_lambda}

Here we define the decay constant values we wish to test, in this case those between $10^{-2}$ and $10^3$, doubling each step.

In [5]:
tau_2_values = [0.01]
while tau_2_values[-1] < 1:
    tau_2_values += [tau_2_values[-1]*2]

tau_2_values

[0.01, 0.02, 0.04, 0.08, 0.16, 0.32, 0.64, 1.28]

Next we run the loop which will generate the data on a tau_2 by tau_2 individual basis for each value in the above code block.

With this data, in each iteration of the loop:

- The data will be prepared for DeepMoD injection
- DeepMoD will try its best
- The results will be organised and saved in a named folder.
- The progress will be available in Tensorboard files also which will need to be manually dragged across after the loop is done (or during!)

In [6]:
for tau_2 in tau_2_values:
    
    # reset randomised initialisation
    np.random.seed(0)
    torch.manual_seed(0)

    
    # DATA GENERATION
    # Include new tau_2 value for this iteration of the loop
    eta[1] = tau_2*E[2]
    
    # generate data
    print('Generating data for tau_2 value:', tau_2)
    strain_array, stress_array = VE_datagen.calculate_strain_stress(input_type, time_array, input_expr, E, eta, D_input_lambda=dsigma)
    
    # reshape data into columns
    time_array = time_array.reshape(-1, 1)
    strain_array = strain_array.reshape(-1, 1)
    
    # randomly sample
    reordered_row_indices = np.random.permutation(time_array.size)
    reduced_time_array = time_array[reordered_row_indices, :][:number_of_samples]
    reduced_strain_array = strain_array[reordered_row_indices, :][:number_of_samples]
    
    
    # DEEPMOD PREPARATION
    # convert to tensors
    time_tensor = torch.tensor(reduced_time_array, dtype=torch.float32, requires_grad=True)
    strain_tensor = torch.tensor(reduced_strain_array, dtype=torch.float32)

    
    # DEEPMOD
    # record start time for later transfer of tensorboard files to correct folders
    now = datetime.now()
    dt_string = now.strftime('%d/%m/%Y %H:%M:%S')
    
    # run DeepMoD
    print('Running DeepMoD')
    sparse_coeff_vector_list_list, scaled_coeff_vector_list_list, sparsity_mask_list_list, network = DeepMoD(time_tensor, strain_tensor, network_config, lib_config, optim_config)
    print('Saving results')

    
    # ORGANISING RESULTS
    # calculate expected coeffs. Depending on input type, a different model will have been used to intepret the provided model parameters during data generation.
    # the choice of model must be taken into account to calculate the coeffs that are correct for the data generated.
    if input_type == 'Stress':
        expected_coeffs = VE_params.coeffs_from_model_params_kelvin(E, eta)
    elif input_type == 'Strain':
        expected_coeffs = VE_params.coeffs_from_model_params_maxwell(E, eta)
    
    # reshape, sample and convert to arrays all series data for homogeneity of form and saving
    stress_array = stress_array.reshape(-1,1)
    reduced_stress_array = stress_array[reordered_row_indices, :][:number_of_samples]
    prediction_array = np.array(network(time_tensor).detach())
    
    # convert list of expected coeffs to array for saving
    target_coeffs_array = np.array(expected_coeffs).reshape(-1,1)
    
    # convert pre-thresholding coeffs data to arrays for saving
    pre_thresh_coeffs_array = np.array(sparse_coeff_vector_list_list[0][0].detach())
    pre_thresh_scaled_coeffs_array = np.array(scaled_coeff_vector_list_list[0][0].detach())
    
    # convert final coeffs data to arrays for saving
    final_coeffs_array = np.array(sparse_coeff_vector_list_list[-1][0].detach())
    final_scaled_coeffs_array = np.array(scaled_coeff_vector_list_list[-1][0].detach())
    sparsity_mask_array = np.array(sparsity_mask_list_list[-1][0]).reshape(-1,1)
    
    # group like data vectors together for saving
    series_data = np.concatenate((reduced_time_array, reduced_strain_array, reduced_stress_array, prediction_array), axis=1)
    pre_thresh_coeffs_data = np.concatenate((pre_thresh_coeffs_array, pre_thresh_scaled_coeffs_array), axis=1)
    final_coeffs_data = np.concatenate((final_coeffs_array, final_scaled_coeffs_array, sparsity_mask_array), axis=1)
    
    # Gather miscellaneous information into lists for saving
    DG_info_list = [str(omega), str(E), str(eta), input_type, func_desc]
    misc_list = [dt_string, investigated_param, str(tau_2)]


    # SAVING RESULTS
    # collect parameters to name folder for saving
    first_subfolder = investigated_param
    second_subfolder = 'param_' + str(tau_2).replace('.', '-')
    parent_folder = '../data/Results_tau2_testing'
    foldername = parent_folder + '/' + first_subfolder + '/' + second_subfolder

    # make folder
    if not os.path.isdir(foldername):
        os.makedirs(foldername)
    
    # save all array data
    np.savetxt(foldername+'/series_data.csv', series_data, delimiter=',', header='Time, Target_Strain, Stress, Prediction_Strain')
    np.savetxt(foldername+'/expected_coeffs.csv', target_coeffs_array, delimiter=',')
    np.savetxt(foldername+'/pre_thresh_coeffs_data.csv', pre_thresh_coeffs_data, delimiter=',', header='Prediction_Coeffs, Scaled_Prediction_Coeffs')
    np.savetxt(foldername+'/final_coeffs_data.csv', final_coeffs_data, delimiter=',', header='Prediction_Coeffs, Scaled_Prediction_Coeffs, Sparsity_Mask')
    
    # save all lists data
    with open(foldername+'/misc_list.txt', 'w') as file:
        file.writelines("%s\n" % line for line in misc_list)

    with open(foldername+'/DG_info_list.txt', 'w') as file:
        file.writelines("%s\n" % line for line in DG_info_list)

Generating data for tau_2 value: 0.01
Running DeepMoD
Epoch | Total loss | MSE | PI | L1 
0 3.3E-01 2.6E-01 6.5E-02 1.9E-04
tensor([[1.3179],
        [0.2597],
        [0.6223],
        [0.2686],
        [0.8649],
        [1.1199],
        [0.5159]], requires_grad=True)
lrs are 0.001 0.001
Time elapsed: 0.0 minutes 0.5257399082183838 seconds
[tensor([[1.2023],
        [0.3730],
        [0.4885],
        [0.3851],
        [0.7605],
        [0.9977],
        [0.6250]], requires_grad=True)] [tensor([[1.2023],
        [0.3730],
        [0.4885],
        [0.3851],
        [0.7605],
        [0.9977],
        [0.6250]], requires_grad=True)] [tensor([0, 1, 2, 3, 4, 5, 6])]
Now running final cycle.
Epoch | Total loss | MSE | PI | L1 
0 1.3E-01 6.6E-02 6.2E-02 0.0E+00
tensor([[1.2023],
        [0.3730],
        [0.4885],
        [0.3851],
        [0.7605],
        [0.9977],
        [0.6250]], requires_grad=True)
lrs are 0.001 0.001
Time elapsed: 0.0 minutes 0.48818016052246094 seconds
Saving res

KeyboardInterrupt: 